<a href="https://colab.research.google.com/github/aha2003/F28WP-lab1/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
dataset_path = "/content/drive/MyDrive/Kaggle_Datasets/sdss-image-dataset"
os.listdir(dataset_path)



['processed_sdss.h5']

In [3]:
import h5py

file_path = f"{dataset_path}/processed_sdss.h5"

with h5py.File(file_path, "r") as f:
    print("Keys in HDF5 file:", list(f.keys()))
    for key in f.keys():
        print(f"{key}: shape={f[key].shape}, dtype={f[key].dtype}")


Keys in HDF5 file: ['ObjID', 'death_status', 'dec', 'e_bv', 'g_mag', 'g_minus_r', 'i_mag', 'i_minus_z', 'images', 'r_mag', 'r_minus_i', 'ra', 'specObjID', 'specz_redshift', 'specz_redshift_err', 'u_mag', 'u_minus_g', 'z_mag']
ObjID: shape=(136611,), dtype=|S20
death_status: shape=(136611,), dtype=|S15
dec: shape=(136611,), dtype=float64
e_bv: shape=(136611,), dtype=float32
g_mag: shape=(136611,), dtype=float32
g_minus_r: shape=(136611,), dtype=float32
i_mag: shape=(136611,), dtype=float32
i_minus_z: shape=(136611,), dtype=float32
images: shape=(136611, 107, 107, 5), dtype=float32
r_mag: shape=(136611,), dtype=float32
r_minus_i: shape=(136611,), dtype=float32
ra: shape=(136611,), dtype=float64
specObjID: shape=(136611,), dtype=int64
specz_redshift: shape=(136611,), dtype=float32
specz_redshift_err: shape=(136611,), dtype=float32
u_mag: shape=(136611,), dtype=float32
u_minus_g: shape=(136611,), dtype=float32
z_mag: shape=(136611,), dtype=float32


In [5]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler


file_path = f"{dataset_path}/processed_sdss.h5"

with h5py.File(file_path, 'r') as f:
    print("Keys in HDF5 file:", list(f.keys()))

    # Target (labels)
    y = np.array(f["death_status"][:]).astype(str)

    # Input features
    X = np.column_stack([
        f["u_mag"][:],
        f["g_mag"][:],
        f["r_mag"][:],
        f["i_mag"][:],
        f["z_mag"][:],
        f["u_minus_g"][:],
        f["g_minus_r"][:],
        f["r_minus_i"][:],
        f["i_minus_z"][:],
        f["specz_redshift"][:],
        f["specz_redshift_err"][:],
        f["e_bv"][:],
        f["ra"][:],
        f["dec"][:]
    ])


Keys in HDF5 file: ['ObjID', 'death_status', 'dec', 'e_bv', 'g_mag', 'g_minus_r', 'i_mag', 'i_minus_z', 'images', 'r_mag', 'r_minus_i', 'ra', 'specObjID', 'specz_redshift', 'specz_redshift_err', 'u_mag', 'u_minus_g', 'z_mag']


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Label classes:", label_encoder.classes_)


Label classes: ['ALIVE' 'DEAD' 'TRANSITIONAL']


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=42
)


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    verbose=1
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7929 - loss: 0.5147 - val_accuracy: 0.9310 - val_loss: 0.1828
Epoch 2/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9209 - loss: 0.1973 - val_accuracy: 0.9713 - val_loss: 0.0895
Epoch 3/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9491 - loss: 0.1322 - val_accuracy: 0.9787 - val_loss: 0.0665
Epoch 4/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9604 - loss: 0.1037 - val_accuracy: 0.9832 - val_loss: 0.0506
Epoch 5/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9682 - loss: 0.0821 - val_accuracy: 0.9839 - val_loss: 0.0449
Epoch 6/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9716 - loss: 0.0742 - val_accuracy: 0.9838 - val_loss: 0.0471
Epoch 7/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9749 - loss: 0.0641 - val_accuracy: 0.9903 - val_loss: 0.0305
Epoch 8/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9768 - loss: 0.0592 - val_accuracy: 0.

In [10]:

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    verbose=1
)


Epoch 1/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6266 - loss: 0.8697 - val_accuracy: 0.7845 - val_loss: 0.5222
Epoch 2/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7836 - loss: 0.5286 - val_accuracy: 0.8475 - val_loss: 0.3871
Epoch 3/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8237 - loss: 0.4266 - val_accuracy: 0.8732 - val_loss: 0.3246
Epoch 4/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8456 - loss: 0.3726 - val_accuracy: 0.8908 - val_loss: 0.2843
Epoch 5/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8618 - loss: 0.3345 - val_accuracy: 0.9010 - val_loss: 0.2540
Epoch 6/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8693 - loss: 0.3085 - val_accuracy: 0.9102 - val_loss: 0.2278
Epoch 7/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8819 - loss: 0.2833 - val_accuracy: 0.9201 - val_loss: 0.2071
Epoch 8/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8937 - loss: 0.2563 - val_accuracy: 0.

In [11]:

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_encoded)), activation='softmax')
]) #Layers with highest accuracy

model.compile(
    optimizer=Adam(learning_rate=5e-4),  #Learning rate with most accuracy (5e-4)
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    verbose=1
)


Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7501 - loss: 0.6248 - val_accuracy: 0.8953 - val_loss: 0.2646
Epoch 2/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8796 - loss: 0.2871 - val_accuracy: 0.9406 - val_loss: 0.1596
Epoch 3/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9231 - loss: 0.1910 - val_accuracy: 0.9708 - val_loss: 0.1079
Epoch 4/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9437 - loss: 0.1472 - val_accuracy: 0.9767 - val_loss: 0.0776
Epoch 5/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9547 - loss: 0.1138 - val_accuracy: 0.9799 - val_loss: 0.0652
Epoch 6/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9624 - loss: 0.0984 - val_accuracy: 0.9841 - val_loss: 0.0542
Epoch 7/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9667 - loss: 0.0893 - val_accuracy: 0.9850 - val_loss: 0.0524
Epoch 8/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9715 - loss: 0.0832 - val_accuracy: 0.9870 - val_

In [12]:


model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    verbose=1
)


Epoch 1/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7204 - loss: 0.6956 - val_accuracy: 0.8791 - val_loss: 0.3233
Epoch 2/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8456 - loss: 0.3677 - val_accuracy: 0.9157 - val_loss: 0.2244
Epoch 3/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8869 - loss: 0.2731 - val_accuracy: 0.9432 - val_loss: 0.1603
Epoch 4/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9169 - loss: 0.2066 - val_accuracy: 0.9611 - val_loss: 0.1202
Epoch 5/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9359 - loss: 0.1671 - val_accuracy: 0.9720 - val_loss: 0.0921
Epoch 6/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9451 - loss: 0.1413 - val_accuracy: 0.9802 - val_loss: 0.0761
Epoch 7/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9557 - loss: 0.1222 - val_accuracy: 0.9795 - val_loss: 0.0677
Epoch 8/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9591 - loss: 0.1082 - val_accuracy: 0.

In [13]:


model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])



model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=128,
    verbose=1
)


Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.7878 - loss: 0.5393 - val_accuracy: 0.9235 - val_loss: 0.1883
Epoch 2/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9051 - loss: 0.2309 - val_accuracy: 0.9664 - val_loss: 0.0926
Epoch 3/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9444 - loss: 0.1447 - val_accuracy: 0.9742 - val_loss: 0.0677
Epoch 4/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9569 - loss: 0.1120 - val_accuracy: 0.9834 - val_loss: 0.0476
Epoch 5/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9648 - loss: 0.0906 - val_accuracy: 0.9807 - val_loss: 0.0481
Epoch 6/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9692 - loss: 0.0810 - val_accuracy: 0.9848 - val_loss: 0.0397
Epoch 7/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9714 - loss: 0.0695 - val_accuracy: 0.9883 - val_loss: 0.0358
Epoch 8/20
673/673 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9751 - loss: 0.0618 - val_accuracy: 0.9830 - val_